In [1]:
import re
from typing import Type
from collections import namedtuple
from math import floor
from itertools import count, cycle
from functools import reduce

In [2]:
with open('../data/2024/day14.txt') as f:
    data = f.read()

In [3]:
grid_height, grid_width = 103, 101
grid_mid_y, grid_mid_x = floor(grid_height/2), floor(grid_width/2)

# (top left, top right, bottom left, bottom right)
quads = (
    ((0,0), (grid_mid_y-1, grid_mid_x-1)),
    ((0,grid_mid_x+1), (grid_mid_y-1, grid_width-1)),
    ((grid_mid_y+1,0), (grid_height-1, grid_mid_x-1)),
    ((grid_mid_y+1,grid_mid_x+1), (grid_height-1,grid_width-1)),
)

In [4]:
Robot: Type = namedtuple('Robot', 'x y vx vy')
pattern = re.compile(r'p=(-?\d+),(-?\d+) v=(-?\d+),(-?\d+)')
robots = [Robot(*map(int, re.match(pattern, specs).groups()))
          for specs in data.split('\n')]

In [5]:
# Use modulo to advance robots to positions at (t) time
def robots_at_t(t: int) -> list:
    robots_prime = []

    for r in robots:
        nx = (r.x + t * r.vx) % grid_width
        ny = (r.y + t * r.vy) % grid_height
        robots_prime.append(Robot(nx, ny, r.vx, r.vy))
    return robots_prime

In [6]:
# Part 1
def quad_counts_at_t(t : int) -> list:
    quad_counts = [list(),list(),list(),list()]

    for robot in robots_at_t(t):
        for i,quad in enumerate(quads):
            # Find the first quad bounds the robot is within
            if (quad[0][1] <= robot.x <= quad[1][1]
                    and quad[0][0] <= robot.y <= quad[1][0]):
                        quad_counts[i].append((robot.y, robot.x))
                        break

    return quad_counts

print("Part 1:", reduce(lambda prod,n: prod*n, map(lambda l: len(l), quad_counts_at_t(100))))

Part 1: 225943500


In [7]:
# Part 2: 6377
from PIL import Image, ImageDraw

images = []
bg_color = (0, 0, 0)
point_color = (255,255,255)

# The cycle was determined by manual pattern analysis of frames
for t in count(start=14, step=101):
    im = Image.new('RGB', (grid_width, grid_height), bg_color)
    draw = ImageDraw.Draw(im)

    for robot in robots_at_t(t):
        draw.point(xy=(robot.x,robot.y), fill=point_color)

    draw.text((100,100), text="t="+str(t), fill=point_color)

    im = im.resize((grid_width*2, grid_height*2), Image.Resampling.LANCZOS)

    images.append(im)

    if t == (14 + 101 * 100): break

images[0].save('./visualizations/day14.gif',
               save_all = True, append_images = images[1:],
               optimize = False, duration = 60)